# PocaFoldAS training (CLI runner)


Wraps the standard training script `scripts/run_training.py` using a config file (no synthetic data generation). Adjust the config path and experiment name, optionally enable Weights & Biases, then launch training.


## 1. Project root


In [ ]:

import os
import pathlib

PROJECT_ROOT = None
for candidate in [pathlib.Path.cwd(), *pathlib.Path.cwd().parents]:
    if (candidate / 'setup.py').exists():
        PROJECT_ROOT = candidate
        break
if PROJECT_ROOT is None:
    raise RuntimeError('Could not locate the repository root. Please run this notebook from within the smlm project.')

os.chdir(PROJECT_ROOT)
print(f'Working directory set to: {PROJECT_ROOT}')



## 2. Select config and optional Weights & Biases
Set `USE_WANDB` to `True` to log the run. The notebook writes a copy of the config with `train.use_wandb` set accordingly under `artifacts/notebook_configs/`. 


In [ ]:
import os
import pathlib
import yaml

CONFIG_PATH = pathlib.Path('configs/config_demo_data.yaml')  # point to your training config
EXP_NAME = 'demo_notebook_run'
LOG_DIR = pathlib.Path(os.getenv('TRAIN_LOG_DIR', '/workspace/smlm_logs'))  # override log directory (set TRAIN_LOG_DIR to change)
USE_WANDB = False  # toggle logging

if not CONFIG_PATH.exists():
    raise FileNotFoundError(f'Config file not found: {CONFIG_PATH}')

# Make a copy with the desired wandb flag and log dir so we do not overwrite the source config
cfg = yaml.safe_load(CONFIG_PATH.read_text())
cfg.setdefault('train', {})
cfg['train']['use_wandb'] = bool(USE_WANDB)
cfg['train']['log_dir'] = str(LOG_DIR)
LOG_DIR.mkdir(parents=True, exist_ok=True)

copy_dir = pathlib.Path('artifacts/notebook_configs')
copy_dir.mkdir(parents=True, exist_ok=True)
CONFIG_COPY = copy_dir / CONFIG_PATH.name
CONFIG_COPY.write_text(yaml.safe_dump(cfg))
print('Using config copy:', CONFIG_COPY.resolve())
print('Log dir:', LOG_DIR.resolve())

if USE_WANDB:
    try:
        import wandb  # type: ignore
        wandb.login()
    except Exception as exc:  # pragma: no cover - convenience in notebook
        print(f'wandb login skipped: {exc}')



### Note on log folder in containers
- Default log dir is `/workspace/smlm_logs` inside the container.
- To change it, set `TRAIN_LOG_DIR` before running the notebook (e.g., `/tmp/smlm_logs`).
- To persist logs on the host, bind-mount a host folder: `-v /host/logs:/workspace/smlm_logs` (or match your `TRAIN_LOG_DIR`).



## 3. Launch training


In [ ]:

import shlex
import subprocess
import sys

cmd = [
    sys.executable, 'scripts/run_training.py',
    '--config', str(CONFIG_COPY),
    '--exp_name', EXP_NAME,
    '--fixed_alpha', '0.001',
]
print('Running:', ' '.join(shlex.quote(c) for c in cmd))
result = subprocess.run(cmd, cwd=PROJECT_ROOT)
print('Exit code:', result.returncode)
if result.returncode != 0:
    raise RuntimeError('Training failed (see logs above).')

